In [ ]:
'''
    @Author: King
    @Date: 2019.03.19
    @Purpose: 中文文本分类实战（六）—— RCNN模型
    @Introduction:  下面介绍
    @Datasets: cnews 中文文本分类数据集。
    @Link : https://github.com/jiangxinyang227/textClassifier/tree/master/RCNN
    @Reference : https://www.cnblogs.com/jiangxinyang/p/10208290.html
'''

## 数据集

cnews 中文文本分类数据集，其目标是判断一段话的类别，数据预处理如文本分类实战（一）—— word2vec预训练词向量中一样，预处理后的文件为../data/cnews_data/labeledCharTrain.csv。


## RCNN 模型结构

RCNN模型来源于论文Recurrent Convolutional Neural Networks for Text Classification {https://arxiv.org/abs/1609.04243}。模型结构图如下：

![avatar](../images/RCNN.png)

　RCNN 整体的模型构建流程如下：

　　1）利用 Bi-LSTM 获得上下文的信息，类似于语言模型。

　　2）将Bi-LSTM获得的隐层输出和词向量拼接[fwOutput, wordEmbedding, bwOutput]。

　　3）将拼接后的向量非线性映射到低维。

　　4）向量中的每一个位置的值都取所有时序上的最大值，得到最终的特征向量，该过程类似于max-pool。

　　5）softmax分类。


In [1]:
import os
import csv
import time
import datetime
import random
import json

import warnings
from collections import Counter
from math import sqrt

import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
warnings.filterwarnings("ignore")

d:\progrom\python\python\python3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
d:\progrom\python\python\python3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 配置参数

class TrainingConfig(object):
    epoches = 10
    evaluateEvery = 10
    checkpointEvery = 10
    learningRate = 0.001
    
class ModelConfig(object):
    embeddingSize = 200
    
    hiddenSizes = [128]  # LSTM结构的神经元个数
    
    dropoutKeepProb = 0.5
    l2RegLambda = 0.0
    
    outputSize = 128  # 从高维映射到低维的神经元个数
    
class Config(object):
    sequenceLength = 200  # 取了所有序列长度的均值
    batchSize = 128
    
    dataSource = "../data/cnews_data/labeledCharTrain.csv"
    
    stopWordSource = "../data/cnews_data/stopwords.txt"
    
    numClasses = 10     #  update
    
    rate = 0.8  # 训练集的比例
    
    training = TrainingConfig()
    
    model = ModelConfig()

    
# 实例化配置参数对象
config = Config()

## 生成训练数据

　　1）将数据加载进来，将句子分割成词表示，并去除低频词和停用词。

　　2）将词映射成索引表示，构建词汇-索引映射表，并保存成json的数据格式，之后做inference时可以用到。（注意，有的词可能不在word2vec的预训练词向量中，这种词直接用UNK表示）

　　3）从预训练的词向量模型中读取出词向量，作为初始化值输入到模型中。

　　4）将数据集分割成训练集和测试集

In [23]:
import sys
# 数据预处理的类，生成训练集和测试集

class Dataset(object):
    def __init__(self, config):
        self._dataSource = config.dataSource
        self._stopWordSource = config.stopWordSource  
        
        self._sequenceLength = config.sequenceLength  # 每条输入的序列处理为定长
        self._embeddingSize = config.model.embeddingSize
        self.numClasses = config.numClasses
        self._batchSize = config.batchSize
        self._rate = config.rate
        
        self._stopWordDict = {}
        
        self.trainReviews = []
        self.trainLabels = []
        
        self.evalReviews = []
        self.evalLabels = []
        
        self.wordEmbedding =None
        
        self._wordToIndex = {}
        self._indexToWord = {}
        self.label_to_index = {}
        self.index_to_label = {}
        
    def _readData(self, filePath):
        """
        从csv文件中读取数据集
        """
        
        df = pd.read_csv(filePath)
        labels = df["class"].tolist()                # update
        review = df["review_cut_word"].tolist()
        reviews = [str(line).strip().split() for line in review]

        return reviews, labels

    def _reviewProcess(self, review, sequenceLength, wordToIndex):
        """
        将数据集中的每条评论用index表示
        wordToIndex中“pad”对应的index为0
        """
        
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in wordToIndex:
                reviewVec[i] = wordToIndex[review[i]]
            else:
                reviewVec[i] = wordToIndex["UNK"]

        return reviewVec

    def _labelProcess(self, y, label_to_index):
        """
        将数据集中的每个label用index表示
        label_to_index“pad”对应的index为0
        """
        input_y = []
        for item in y:
            temp = [0] * self.numClasses
            temp[label_to_index[item]] = 1
            input_y.append(temp)
        
        return input_y
    
    def _genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        
        reviews = []
        labels = []
        
        input_y = self._labelProcess(y,self.label_to_index)
#         print("input_y:{0}".format(input_y[0:2]))
        
#         sys.exit(0)
        
        # 遍历所有的文本，将文本中的词转换成index表示
        for i in range(len(x)):
            reviewVec = self._reviewProcess(x[i], self._sequenceLength, self._wordToIndex)
            reviews.append(reviewVec)
            
            labels.append(input_y[i])
            
        trainIndex = int(len(x) * rate)
        
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")

        return trainReviews, trainLabels, evalReviews, evalLabels
        
    def _genVocabulary(self, reviews,labels):
        """
        生成词向量和词汇-索引映射字典，可以用全数据集
        """
        
        allWords = [word for review in reviews for word in review]
        
        # 去掉停用词
        subWords = [word for word in allWords if word not in self.stopWordDict]
        
        wordCount = Counter(subWords)  # 统计词频
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        
        # 去除低频词
        words = [item[0] for item in sortWordCount if item[1] >= 5]
        
        vocab, wordEmbedding = self._getWordEmbedding(words)
        self.wordEmbedding = wordEmbedding
        
        self._wordToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self._indexToWord = dict(zip(list(range(len(vocab))), vocab))
        

        labels = set(labels)

        for i, label in enumerate(labels):
            self.label_to_index[label] = i
            self.index_to_label[int(i)] = label
        
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../data/wordJson/wordToIndex.json", "w", encoding="utf-8") as f:
            json.dump(self._wordToIndex, f)
        
        with open("../data/wordJson/indexToWord.json", "w", encoding="utf-8") as f:
            json.dump(self._indexToWord, f)
            
        with open('../data/wordJson/label_to_index.json', 'w', encoding='utf-8') as f:
            json.dump(self.label_to_index, f, ensure_ascii=False)

        with open('../data/wordJson/index_to_label.json', 'w', encoding='utf-8') as f:
            json.dump(self.index_to_label, f, ensure_ascii=False)
            
    def _getWordEmbedding(self, words):
        """
        按照我们的数据集中的单词取出预训练好的word2vec中的词向量
        """
        
        wordVec = gensim.models.KeyedVectors.load_word2vec_format("../data/ChnSentiCorp_cn_data/word2Vec.bin", binary=True)
        vocab = []
        wordEmbedding = []
        
        # 添加 "pad" 和 "UNK", 
        vocab.append("pad")
        vocab.append("UNK")
        wordEmbedding.append(np.zeros(self._embeddingSize))
        wordEmbedding.append(np.random.randn(self._embeddingSize))
        
        for word in words:
            try:
                vector = wordVec.wv[word]
                vocab.append(word)
                wordEmbedding.append(vector)
            except:
                #print(word + "不存在于词向量中")
                pass
                
        return vocab, np.array(wordEmbedding)
    
    def _readStopWord(self, stopWordPath):
        """
        读取停用词
        """
        
        with open(stopWordPath, "r", encoding="utf-8") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            # 将停用词用列表的形式生成，之后查找停用词时会比较快
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
            
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        
        # 初始化停用词
        self._readStopWord(self._stopWordSource)
        
        # 初始化数据集
        reviews, labels = self._readData(self._dataSource)
        
        # 初始化词汇-索引映射表和词向量矩阵
        self._genVocabulary(reviews,labels)
        
        # 初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviews, labels, self._rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        print("self.trainReviews[0:2]:{0}".format(self.trainReviews[0:3]))
        print("self.trainLabels[0:2]:{0}".format(self.trainLabels[0:3]))
        
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels
        print("self.evalReviews[0:2]:{0}".format(self.evalReviews[0:3]))
        print("self.evalLabels[0:2]:{0}".format(self.evalLabels[0:3]))
        
        
data = Dataset(config)
data.dataGen()

self.trainReviews[0:2]:[[   1    1    1    1    1    1    1    1    1    1    1    1 1131    1
     1    1    1  478  142    1   27    2    1  483    1    1    1    1
     1    1 1281    1    1    1    1    1    1  886    1    1    1    1
     1    1    1    1    1    1 1005    1    1    1   13    1    1    1
     1    1    1    1    1    1    1  203  215    1    1    1    1    1
     1    1   27    1    1    1    1    1  102    1  152    1    1    1
     1  451    1    1  753    1    1  226    1    1   65    1    1    1
     1    1    1   19    1    1    1    1    1  849    1    1 2243    1
     1    1    1    1    1    1    1    1  878    1    1    1    1    1
     1    1    1    1    1    1    1    1    1    1    1  419    1    1
   196    1    1    1    1    1    1    1    1    1    1    1    1    1
     1    1   10    1    1    8    1    1 1654    1    1    1    1    1
     1    1    1   36    1    1    1 1151    1    1    1    1    1    1
     1    1  120    1    1 1162    1    1

In [24]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("eval data shape: {}".format(data.evalReviews.shape))

train data shape: (3999, 200)
train label shape: (3999, 10)
eval data shape: (1000, 200)


## 生成batch数据集

　　采用生成器的形式向模型输入batch数据集，（生成器可以避免将所有的数据加入到内存中）

In [1]:
# 输出batch数据集

def nextBatch(x, y, batchSize):
        """
        生成batch数据集，用生成器的方式输出
        """
    
        perm = np.arange(len(x))
        np.random.shuffle(perm)
        x = x[perm]
        y = y[perm]
        
        numBatches = len(x) // batchSize

        for i in range(numBatches):
            start = i * batchSize
            end = start + batchSize
            batchX = np.array(x[start: end], dtype="int64")
            batchY = np.array(y[start: end], dtype="float32")
            
            yield batchX, batchY

In [26]:
"""
构建模型，模型的架构如下：
1，利用Bi-LSTM获得上下文的信息
2，将Bi-LSTM获得的隐层输出和词向量拼接[fwOutput;wordEmbedding;bwOutput]
3，将2所得的词表示映射到低维
4，hidden_size上每个位置的值都取时间步上最大的值，类似于max-pool
5，softmax分类
"""

class RCNN(object):
    """
    RCNN 用于文本分类
    """
    def __init__(self, config, wordEmbedding):

        # 定义模型的输入
        self.inputX = tf.placeholder(tf.int32, [None, config.sequenceLength], name="inputX")
        self.numClasses = config.numClasses
        self.inputY = tf.placeholder(tf.float32, [None, self.numClasses], name="inputY")   # update
        
        
        self.dropoutKeepProb = tf.placeholder(tf.float32, name="dropoutKeepProb")
        
        # 定义l2损失
        l2Loss = tf.constant(0.0)
        
        # 词嵌入层
        with tf.name_scope("embedding"):

            # 利用预训练的词向量初始化词嵌入矩阵
            self.W = tf.Variable(tf.cast(wordEmbedding, dtype=tf.float32, name="word2vec") ,name="W")
            # 利用词嵌入矩阵将输入的数据中的词转换成词向量，维度[batch_size, sequence_length, embedding_size]
            self.embeddedWords = tf.nn.embedding_lookup(self.W, self.inputX)
            # 复制一份embedding input
            self.embeddedWords_ = self.embeddedWords
            
        # 定义两层双向LSTM的模型结构
        with tf.name_scope("Bi-LSTM"):
            for idx, hiddenSize in enumerate(config.model.hiddenSizes):
                with tf.name_scope("Bi-LSTM" + str(idx)):
                    # 定义前向LSTM结构
                    lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)
                    # 定义反向LSTM结构
                    lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)


                    # 采用动态rnn，可以动态的输入序列的长度，若没有输入，则取序列的全长
                    # outputs是一个元祖(output_fw, output_bw)，其中两个元素的维度都是[batch_size, max_time, hidden_size],fw和bw的hidden_size一样
                    # self.current_state 是最终的状态，二元组(state_fw, state_bw)，state_fw=[batch_size, s]，s是一个元祖(h, c)
                    outputs_, self.current_state = tf.nn.bidirectional_dynamic_rnn(lstmFwCell, lstmBwCell, 
                                                                                  self.embeddedWords_, dtype=tf.float32,
                                                                                  scope="bi-lstm" + str(idx))
        
                    # 对outputs中的fw和bw的结果拼接 [batch_size, time_step, hidden_size * 2], 传入到下一层Bi-LSTM中
                    self.embeddedWords_ = tf.concat(outputs_, 2)
                
        # 将最后一层Bi-LSTM输出的结果分割成前向和后向的输出
        fwOutput, bwOutput = tf.split(self.embeddedWords_, 2, -1)
            
        with tf.name_scope("context"):
            shape = [tf.shape(fwOutput)[0], 1, tf.shape(fwOutput)[2]]
            self.contextLeft = tf.concat([tf.zeros(shape), fwOutput[:, :-1]], axis=1, name="contextLeft")
            self.contextRight = tf.concat([bwOutput[:, 1:], tf.zeros(shape)], axis=1, name="contextRight")
            
        # 将前向，后向的输出和最早的词向量拼接在一起得到最终的词表征
        with tf.name_scope("wordRepresentation"):
            self.wordRepre = tf.concat([self.contextLeft, self.embeddedWords, self.contextRight], axis=2, name='x')
            wordSize = config.model.hiddenSizes[-1] * 2 + config.model.embeddingSize 
        
        with tf.name_scope("textRepresentation"):
            outputSize = config.model.outputSize
            textW = tf.Variable(tf.random_uniform([wordSize, outputSize], -1.0, 1.0), name="W2")
            textB = tf.Variable(tf.constant(0.1, shape=[outputSize]), name="b2")
            
            # tf.einsum可以指定维度的消除运算
            self.textRepre = tf.tanh(tf.einsum('aij,jk->aik', self.wordRepre, textW) + textB)
            
        # 做max-pool的操作，将时间步的维度消失
        output = tf.reduce_max(self.textRepre, axis=1)
        
        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW",
                shape=[outputSize, self.numClasses],
                initializer=tf.contrib.layers.xavier_initializer())
            
            outputB= tf.Variable(tf.constant(0.1, shape=[self.numClasses]), name="outputB")
            l2Loss += tf.nn.l2_loss(outputW)
            l2Loss += tf.nn.l2_loss(outputB)
#             self.predictions = tf.nn.xw_plus_b(output, outputW, outputB, name="predictions")
#             self.binaryPreds = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
            self.scores = tf.nn.xw_plus_b(output, outputW, outputB, name='scores')
            self.predictions = tf.argmax(self.scores, 1, name='predictions')
        
        # 计算二元交叉熵损失
        with tf.name_scope("loss"):
            
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.inputY)
            self.loss = tf.reduce_mean(losses) + config.model.l2RegLambda * l2Loss
            
        # accuracy
        with tf.name_scope('accuracy'):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.inputY, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32), name='accuracy')

In [27]:
# 定义性能指标函数

def mean(item):
    return sum(item) / len(item)


def genMetrics(trueY, predY, binaryPredY):
    """
    生成acc和auc值
    """
    auc = 0
    try:
        auc = roc_auc_score(trueY, predY)
    except ValueError:
        pass
    accuracy = accuracy_score(trueY, binaryPredY)
    precision = precision_score(trueY, binaryPredY)
    recall = recall_score(trueY, binaryPredY)
    
    return round(accuracy, 4), round(auc, 4), round(precision, 4), round(recall, 4)

In [42]:
# 训练模型

# 生成训练集和验证集
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels

wordEmbedding = data.wordEmbedding


# 定义计算图
with tf.Graph().as_default():

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth=True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置gpu占用率  

    sess = tf.Session(config=session_conf)
    
    # 定义会话
    with sess.as_default():
        lstm = RCNN(config, wordEmbedding)
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(config.training.learningRate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(lstm.loss)
        # 将梯度应用到变量下，生成训练器
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        # 用summary绘制tensorBoard
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
        print("Writing to {}\n".format(outDir))
        
        lossSummary = tf.summary.scalar("loss", lstm.loss)
        acc_summary = tf.summary.scalar('accuracy', lstm.accuracy)
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter = tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        
        # 初始化所有变量
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        
        # 保存模型的一种方式，保存为pb文件
        builder = tf.saved_model.builder.SavedModelBuilder("../model/RCNN/savedModel")
        sess.run(tf.global_variables_initializer())

        def trainStep(batchX, batchY):
            """
            训练函数
            """   
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: config.model.dropoutKeepProb
            }
            _, summaries,step, loss, accuracy = sess.run(
                [trainOp, summaryOp, globalStep, lstm.loss, lstm.accuracy],
                feed_dict=feed_dict
            )
            timeStr = datetime.datetime.now().isoformat()
            #acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            #print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(timeStr, step, loss, acc, auc, precision, recall))
            print("{}: step {}, loss {:g}, acc {:g}".format(timeStr, step, loss, accuracy))
            trainSummaryWriter.add_summary(summaries, step)

        def devStep(batchX, batchY, writer=None):
            """
            验证函数
            """
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: 1.0
            }
            _, summaries,step, loss, accuracy = sess.run(
                [trainOp, summaryOp, globalStep, lstm.loss, lstm.accuracy],
                feed_dict=feed_dict
            )
            
            #acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            timeStr = datetime.datetime.now().isoformat()
            print("Evaluation:{}: step {}, loss {:g}, acc {:g}".format(timeStr, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)
            
        
        for i in range(config.training.epoches):
            # 训练模型
            print("start training model")
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                trainStep(batchTrain[0], batchTrain[1])

                currentStep = tf.train.global_step(sess, globalStep) 
                if currentStep % config.training.evaluateEvery == 0:
                    print('Evaluation:')
                    losses = []
                    accs = []
                    aucs = []
                    precisions = []
                    recalls = []
                    for batchEval in nextBatch(evalReviews, evalLabels, config.batchSize):
                        devStep(evalReviews, evalLabels, writer=evalSummaryWriter)
                
                
                if currentStep % config.training.checkpointEvery == 0:
                    # 保存模型的另一种方法，保存checkpoint文件
                    path = saver.save(sess, "../model/RCNN/model/my-model", global_step=currentStep)
                    print("Saved model checkpoint to {}\n".format(path))
                    
#         inputs = {"inputX": tf.saved_model.utils.build_tensor_info(lstm.inputX),
#                   "keepProb": tf.saved_model.utils.build_tensor_info(lstm.dropoutKeepProb)}

#         outputs = {"binaryPreds": tf.saved_model.utils.build_tensor_info(lstm.binaryPreds)}

#         prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, outputs=outputs,
#                                                                                       method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
#         legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
#         builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
#                                             signature_def_map={"predict": prediction_signature}, legacy_init_op=legacy_init_op)

#         builder.save()

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/kernel:0/grad/hist is illegal; using bi-lstm0/fw/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/kernel:0/grad/sparsity is illegal; using bi-lstm0/fw/lstm_cell/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/bias:0/grad/hist is illegal; using bi-lstm0/fw/lstm_cell/bias_0/grad/hist instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/bias:0/grad/sparsity is illegal; using bi-lstm0/fw/lstm_cell/bias_0/grad/sparsity instead.
INFO:tensorflow:Summary name bi-lstm0/bw/lstm_cell/kernel:0/grad/hist is illegal; using bi-lstm0/bw/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name bi-lstm0/bw/lstm_cell/kernel:0/grad/sparsity is illegal; 

2019-04-17T09:51:02.978297: step 79, loss 2.1534, acc 0.195312
2019-04-17T09:51:05.345181: step 80, loss 2.25387, acc 0.0859375
Evaluation:
Evaluation:2019-04-17T09:51:17.172714: step 81, loss 0.217195, acc 0.987
Evaluation:2019-04-17T09:51:29.226299: step 82, loss 0.213737, acc 0.988
Evaluation:2019-04-17T09:51:41.315624: step 83, loss 0.209142, acc 0.989
Evaluation:2019-04-17T09:51:53.181722: step 84, loss 0.20357, acc 0.989
Evaluation:2019-04-17T09:52:05.068218: step 85, loss 0.197556, acc 0.989
Evaluation:2019-04-17T09:52:17.175782: step 86, loss 0.191491, acc 0.991
Evaluation:2019-04-17T09:52:29.462712: step 87, loss 0.185828, acc 0.993
Saved model checkpoint to ../model/RCNN/model/my-model-80

start training model
2019-04-17T09:52:32.283333: step 88, loss 2.20375, acc 0.140625
2019-04-17T09:52:34.727592: step 89, loss 2.16753, acc 0.148438
2019-04-17T09:52:36.994728: step 90, loss 2.10982, acc 0.21875
Evaluation:
Evaluation:2019-04-17T09:52:49.249087: step 91, loss 0.170966, acc 

Evaluation:2019-04-17T10:07:47.394668: step 184, loss 0.0586883, acc 0.999
Evaluation:2019-04-17T10:07:59.998959: step 185, loss 0.0577058, acc 0.999
Evaluation:2019-04-17T10:08:12.953257: step 186, loss 0.0567007, acc 0.999
Evaluation:2019-04-17T10:08:26.060358: step 187, loss 0.0556316, acc 0.999
Saved model checkpoint to ../model/RCNN/model/my-model-180

2019-04-17T10:08:29.110325: step 188, loss 2.09856, acc 0.132812
start training model
2019-04-17T10:08:31.441189: step 189, loss 2.09675, acc 0.109375
2019-04-17T10:08:33.943985: step 190, loss 2.05843, acc 0.15625
Evaluation:
Evaluation:2019-04-17T10:08:47.210555: step 191, loss 0.0521762, acc 0.999
Evaluation:2019-04-17T10:09:00.487272: step 192, loss 0.0515419, acc 0.999
Evaluation:2019-04-17T10:09:13.810901: step 193, loss 0.0508405, acc 1
Evaluation:2019-04-17T10:09:26.994811: step 194, loss 0.050059, acc 1
Evaluation:2019-04-17T10:09:40.152190: step 195, loss 0.0492197, acc 1
Evaluation:2019-04-17T10:09:52.891931: step 196, lo

Evaluation:2019-04-17T10:26:08.110705: step 291, loss 0.0208665, acc 1
Evaluation:2019-04-17T10:26:21.826374: step 292, loss 0.0207436, acc 1
Evaluation:2019-04-17T10:26:35.857523: step 293, loss 0.0206178, acc 1
Evaluation:2019-04-17T10:26:49.992386: step 294, loss 0.0204821, acc 1
Evaluation:2019-04-17T10:27:04.225963: step 295, loss 0.0203334, acc 1
Evaluation:2019-04-17T10:27:17.895400: step 296, loss 0.0201715, acc 1
Evaluation:2019-04-17T10:27:31.795309: step 297, loss 0.0199991, acc 1
Saved model checkpoint to ../model/RCNN/model/my-model-290

2019-04-17T10:27:35.053884: step 298, loss 1.95316, acc 0.265625
2019-04-17T10:27:37.368869: step 299, loss 1.99333, acc 0.242188
2019-04-17T10:27:39.627712: step 300, loss 1.9698, acc 0.273438
Evaluation:
Evaluation:2019-04-17T10:27:53.914309: step 301, loss 0.0194725, acc 1
Evaluation:2019-04-17T10:28:08.660596: step 302, loss 0.0193973, acc 1
Evaluation:2019-04-17T10:28:22.310922: step 303, loss 0.0192962, acc 1
Evaluation:2019-04-17T10

2019-04-17T10:45:22.410678: step 398, loss 1.87138, acc 0.265625
2019-04-17T10:45:24.726592: step 399, loss 1.88569, acc 0.335938
2019-04-17T10:45:27.024246: step 400, loss 1.90455, acc 0.273438
Evaluation:
Evaluation:2019-04-17T10:45:41.710348: step 401, loss 0.0129946, acc 1
Evaluation:2019-04-17T10:45:56.178065: step 402, loss 0.0130112, acc 1
Evaluation:2019-04-17T10:46:10.317413: step 403, loss 0.0130488, acc 1
Evaluation:2019-04-17T10:46:24.510633: step 404, loss 0.0130774, acc 1
Evaluation:2019-04-17T10:46:38.974939: step 405, loss 0.0130838, acc 1
Evaluation:2019-04-17T10:46:53.273061: step 406, loss 0.0130629, acc 1
Evaluation:2019-04-17T10:47:07.714542: step 407, loss 0.0130176, acc 1
Saved model checkpoint to ../model/RCNN/model/my-model-400

2019-04-17T10:47:10.741881: step 408, loss 1.83989, acc 0.351562
2019-04-17T10:47:13.321797: step 409, loss 1.89568, acc 0.25
2019-04-17T10:47:15.685955: step 410, loss 1.84373, acc 0.375
Evaluation:
Evaluation:2019-04-17T10:47:30.01094

Evaluation:2019-04-17T11:05:11.395055: step 507, loss 0.0112484, acc 1
Saved model checkpoint to ../model/RCNN/model/my-model-500

2019-04-17T11:05:14.446526: step 508, loss 1.68751, acc 0.445312
2019-04-17T11:05:17.095129: step 509, loss 1.59752, acc 0.492188
2019-04-17T11:05:19.485370: step 510, loss 1.69005, acc 0.40625
Evaluation:
Evaluation:2019-04-17T11:05:33.983157: step 511, loss 0.0111226, acc 1
Evaluation:2019-04-17T11:05:48.670520: step 512, loss 0.0111897, acc 1
Evaluation:2019-04-17T11:06:02.903276: step 513, loss 0.0112668, acc 1
Evaluation:2019-04-17T11:06:15.707325: step 514, loss 0.0113369, acc 1
Evaluation:2019-04-17T11:06:30.157009: step 515, loss 0.011387, acc 1
Evaluation:2019-04-17T11:06:44.710875: step 516, loss 0.0114026, acc 1
Evaluation:2019-04-17T11:06:59.290659: step 517, loss 0.01138, acc 1
Saved model checkpoint to ../model/RCNN/model/my-model-510

2019-04-17T11:07:02.226377: step 518, loss 1.56771, acc 0.570312
2019-04-17T11:07:04.535036: step 519, loss 1

Evaluation:2019-04-17T11:24:27.037397: step 614, loss 0.0110309, acc 1
Evaluation:2019-04-17T11:24:41.563906: step 615, loss 0.0112194, acc 1
Evaluation:2019-04-17T11:24:55.950717: step 616, loss 0.0113101, acc 1
Evaluation:2019-04-17T11:25:10.364433: step 617, loss 0.0113004, acc 1
Saved model checkpoint to ../model/RCNN/model/my-model-610

2019-04-17T11:25:13.458791: step 618, loss 1.2868, acc 0.609375
2019-04-17T11:25:15.576956: step 619, loss 1.20474, acc 0.6875
2019-04-17T11:25:17.858544: step 620, loss 1.21706, acc 0.742188
Evaluation:
Evaluation:2019-04-17T11:25:31.244322: step 621, loss 0.0104643, acc 1
Evaluation:2019-04-17T11:25:45.580562: step 622, loss 0.0104617, acc 1
Evaluation:2019-04-17T11:25:59.783353: step 623, loss 0.0105031, acc 1
Evaluation:2019-04-17T11:26:14.471555: step 624, loss 0.010533, acc 1
Evaluation:2019-04-17T11:26:29.071088: step 625, loss 0.0105148, acc 1
Evaluation:2019-04-17T11:26:43.363820: step 626, loss 0.0104349, acc 1
Evaluation:2019-04-17T11:26

Evaluation:2019-04-17T11:43:42.495450: step 721, loss 0.00874627, acc 1
Evaluation:2019-04-17T11:43:56.879912: step 722, loss 0.00833857, acc 1
Evaluation:2019-04-17T11:44:11.350153: step 723, loss 0.00800485, acc 1
Evaluation:2019-04-17T11:44:25.927886: step 724, loss 0.00772815, acc 1
Evaluation:2019-04-17T11:44:40.761159: step 725, loss 0.00749194, acc 1
Evaluation:2019-04-17T11:44:55.110610: step 726, loss 0.00728562, acc 1
Evaluation:2019-04-17T11:45:09.166516: step 727, loss 0.00709989, acc 1
Saved model checkpoint to ../model/RCNN/model/my-model-720

2019-04-17T11:45:12.394976: step 728, loss 0.923749, acc 0.804688
2019-04-17T11:45:14.785819: step 729, loss 0.818303, acc 0.859375
2019-04-17T11:45:17.096549: step 730, loss 0.741281, acc 0.882812
Evaluation:
Evaluation:2019-04-17T11:45:31.936134: step 731, loss 0.00683651, acc 1
Evaluation:2019-04-17T11:45:46.750542: step 732, loss 0.00703646, acc 1
Evaluation:2019-04-17T11:46:01.244901: step 733, loss 0.00727836, acc 1
Evaluation

Saved model checkpoint to ../model/RCNN/model/my-model-820

2019-04-17T12:03:29.936624: step 828, loss 0.677504, acc 0.804688
2019-04-17T12:03:32.294892: step 829, loss 0.579982, acc 0.875
2019-04-17T12:03:34.595284: step 830, loss 0.677371, acc 0.820312
Evaluation:
Evaluation:2019-04-17T12:03:49.478906: step 831, loss 0.00875128, acc 1
Evaluation:2019-04-17T12:04:04.990964: step 832, loss 0.00942291, acc 1
Evaluation:2019-04-17T12:04:21.549670: step 833, loss 0.0100995, acc 1
Evaluation:2019-04-17T12:04:37.320403: step 834, loss 0.0107135, acc 1
Evaluation:2019-04-17T12:04:52.910468: step 835, loss 0.0111814, acc 1
Evaluation:2019-04-17T12:05:08.547465: step 836, loss 0.0114208, acc 1
Evaluation:2019-04-17T12:05:23.881813: step 837, loss 0.0113763, acc 1
Saved model checkpoint to ../model/RCNN/model/my-model-830

2019-04-17T12:05:26.625944: step 838, loss 0.667177, acc 0.84375
2019-04-17T12:05:28.996450: step 839, loss 0.566256, acc 0.875
2019-04-17T12:05:31.310861: step 840, loss 0.5

Evaluation:2019-04-17T12:23:57.283687: step 934, loss 0.00795117, acc 1
Evaluation:2019-04-17T12:24:12.512275: step 935, loss 0.0075904, acc 1
Evaluation:2019-04-17T12:24:28.780512: step 936, loss 0.00722952, acc 1
Evaluation:2019-04-17T12:24:42.394905: step 937, loss 0.00688042, acc 1
Saved model checkpoint to ../model/RCNN/model/my-model-930

2019-04-17T12:24:45.211002: step 938, loss 0.440015, acc 0.921875
2019-04-17T12:24:47.864195: step 939, loss 0.438189, acc 0.90625
2019-04-17T12:24:50.112062: step 940, loss 0.416066, acc 0.9375
Evaluation:
Evaluation:2019-04-17T12:25:03.919352: step 941, loss 0.00618045, acc 1
Evaluation:2019-04-17T12:25:16.962463: step 942, loss 0.00622002, acc 1
Evaluation:2019-04-17T12:25:30.685146: step 943, loss 0.00625144, acc 1
Evaluation:2019-04-17T12:25:43.694785: step 944, loss 0.00626499, acc 1
Evaluation:2019-04-17T12:25:56.760849: step 945, loss 0.00625235, acc 1
Evaluation:2019-04-17T12:26:12.045209: step 946, loss 0.00620922, acc 1
Evaluation:201